In [2]:
import torch
import numpy as np
import torch.nn as nn

In [24]:
import pandas as pd
import numpy as np

In [25]:
data = pd.read_csv('/Users/abhijitdeshpande/Documents/Project Files/Full Hourly Data/JPM.csv')

In [26]:
label = data.out

In [27]:
import xgboost

In [144]:
from sklearn.svm import SVC

In [162]:
svm = SVC(C=10)

In [163]:
svm.fit(x,label)

SVC(C=10)

In [164]:
pred = svm.predict(x)

In [165]:
accuracy_score(pred,label)

0.5849111564229537

In [166]:
from sklearn.metrics import classification_report, accuracy_score

In [167]:
print(classification_report(label,pred))

              precision    recall  f1-score   support

           0       0.58      1.00      0.74      4016
           1       0.00      0.00      0.00      2850

    accuracy                           0.58      6866
   macro avg       0.29      0.50      0.37      6866
weighted avg       0.34      0.58      0.43      6866



/Users/abhijitdeshpande/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/abhijitdeshpande/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/abhijitdeshpande/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [168]:
import dask

In [169]:
from dask.distributed import Client

In [4]:
client = Client(n_workers=4)

/Users/abhijitdeshpande/opt/anaconda3/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49873 instead
  warnings.warn(


In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:49873/status,
Dashboard: http://127.0.0.1:49873/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49874,Workers: 4
Dashboard: http://127.0.0.1:49873/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:49885,Total threads: 2
Dashboard: http://127.0.0.1:49889/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:49876,


In [82]:
from dask import delayed
import numpy as np

In [46]:
from time import sleep

In [73]:
@delayed
def inc(x):
    sleep(1)
    return x+1

In [74]:
data = [1,2,3,4,5,6,7,8]

In [75]:
%%time

results = []

for i in data:
    y = inc(i)
    results.append(y)
    
total = delayed(sum)(results)

results = total.compute()

CPU times: user 51 ms, sys: 12.1 ms, total: 63.1 ms
Wall time: 1.02 s


In [91]:
z = delayed(np.arange(10))

In [93]:
from dask import compute

In [97]:
compute(z.min())

(0,)

In [104]:
compute((z**2).max())

(81,)

In [118]:
import dask.array as da

In [134]:
x = da.from_array(data,chunks=100)

In [135]:
x

dask.array<array, shape=(6866, 46), dtype=float64, chunksize=(100, 46), chunktype=numpy.ndarray>

In [194]:
compute(x.sum())

(261104018988.82303,)

In [195]:
np.sum(data).sum()

261104018988.823

In [2]:
import scrapy

In [5]:
scrapy.Spider

scrapy.spiders.Spider

In [2]:
import torch.nn as nn
import torch

In [8]:
nn.Conv1d(64,kernel_size=1,stride=(1,1))

TypeError: __init__() missing 1 required positional argument: 'out_channels'

In [5]:
def identity_block(X, f, filters):
    F1, F2, F3 = filters
    
    X_shortcut = X
    cache = []
    
    X = nn.Conv1d(filters=F1, kernel_size=1, strides=(1,1), padding='valid')
    X = nn.BatchNorm1d()(X)
    X = nn.ReLU()(X)
    
    X = nn.Conv1d(filters=F2, kernel_size=f, strides=(1,1), padding='valid')
    X = nn.BatchNorm1d()(X)
    X = nn.ReLU()(X)
    

In [6]:
def convolution_block(X, f, filters, s=2):

    F1, F2, F3 = filters
    
    x_shortcut = X
    
    X = nn.Conv1d(filters=F1, kernel_size=1, strides = (s,s), padding='valid')(X)
    X = nn.ReLU()(X)
    X = nn.BactNorm1d()(X)
    X = nn.ReLU()(X)
   
    X = nn.Conv1d(filters=F2, kernel_size=f, strides = (1,1), padding='same')(X)
    X = nn.ReLU()(X)
    X = nn.BactNorm1d()(X)
    X = nn.ReLU()(X)
    
    X = nn.Conv1d(filters=F3, kernel_size=1, strides = (1,1), padding='valid')(X)
    X = nn.ReLU()(X)
    X = nn.BactNorm1d()(X)
    X = nn.ReLU()(X)
    
    x_shortcut = nn.Conv1d(filters=F3, kernel_size=1, strides=(s,s), padding='valid')(x_shortcut)
    x_shortcut = nn.BatchNorm1d()(x_shortcut)
    
    X = torch.Add(X ,x_shortcut)
    X = nn.ReLU()(X)
    
    return X

In [7]:
def ResNet50(X,classes=2):
    
    X  = nn.Conv1d(15,(7,7),strides=(2,2))(X)
    X = nn.BatchNorm1d()(X)
    X = nn.ReLU()(X)
    X = nn.MaxPool1d((3,3), strides=(2,2))(X)
    
    X = convolution_block(X, f=3, filters = [64,64,256], s=1)
    X

In [63]:
nn.Conv1d(15,64,5,stride=2, padding='valid')

Conv1d(15, 64, kernel_size=(5,), stride=(2,), padding=valid)

In [67]:
class ResBlock(nn.Module):
    def __init__(self, ni, nf, kernels=[7, 5, 3],s=2):
        super(ResBlock,self).__init__()
        
        F1,F2, F3 = kernels
        
        self.conv1 = nn.Conv1d(ni, nf, kernel_size=F1, stride=s,padding='valid')
        self.conv2 = nn.Conv1d(nf, nf, kernel_size=F2,stride=1,padding='same')
        self.conv3 = nn.Conv1d(nf, nf, kernel_size=F3,strdie=1,padding='valid')

        # expand channels for the sum if necessary
        self.shortcut = nn.BatchNorm1d(ni) if ni == nf else nn.Conv1d(ni, nf, 1, stride=2,padding='valid')
        self.add = torch.add
        self.act = nn.ReLU()

    def forward(self, x):
        res = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.add(x, self.shortcut(res))
        x = self.act(x)
        return x
    
class ResNet(nn.Module):
    def __init__(self, c_in, c_out):
        super(ResNet,self).__init__()
        nf = 64
        kernels=[7, 5, 3]
        self.resblock1 = ResBlock(c_in, nf, kernels=kernels)
        self.resblock2 = ResBlock(nf, nf * 2, kernels=kernels)
        self.resblock3 = ResBlock(nf * 2, nf * 2, kernels=kernels)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(nf * 2, c_out)

    def forward(self, x):
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.squeeze(self.gap(x))
        return self.fc(x)

In [60]:
resnet = ResNet(15,2)

In [215]:
from nltk.tokenize import TweetTokenizer
import nltk

In [216]:
tweet= "Snow White and the Seven Degrees #MakeAMovieCold@midnight:-)"

In [217]:
token = TweetTokenizer()

In [218]:
clean_text = token.tokenize(tweet.lower())

In [253]:
pos_text = nltk.pos_tag(clean_text)

In [219]:
def n_grams(text,n):
    
    return [text[i:i+n] for i in range(len(text)-n+1)]

In [220]:
n_grams(clean_text,5)

[['snow', 'white', 'and', 'the', 'seven'],
 ['white', 'and', 'the', 'seven', 'degrees'],
 ['and', 'the', 'seven', 'degrees', '#makeamoviecold'],
 ['the', 'seven', 'degrees', '#makeamoviecold', '@midnight'],
 ['seven', 'degrees', '#makeamoviecold', '@midnight', ':-)']]

In [233]:
from nltk import stem

In [244]:
nltk.pos_tag(token.tokenize('Mary slapped the green witch.'))

[('Mary', 'NNP'),
 ('slapped', 'VBD'),
 ('the', 'DT'),
 ('green', 'JJ'),
 ('witch', 'NN'),
 ('.', '.')]

In [13]:
class train_model():
    
    def __init__(self,model):
        self.model = model
        self.validation_loss = []
        self.training_loss = []
        
    def compile(self, loss=nn.CrossEntropyLoss(),optimizer=torch.optim.Adam,lr=1e-3):  
        self.loss_module = loss
        self.optimizer = optimizer(self.model.parameters(),lr=lr)

   
    def fit(self, data, batch_size ,verbose):

        self.model = self.model.train()
        self.data = data
        total_loss = 0
        total_batch_accuracy = 0

        for i, batch in enumerate(DataLoader(self.data, batch_size)):
            x_train, y_train, masks = batch
            prediction = self.model(x_train.float(),masks)
            #prob = nn.Softmax(prediction)
            #optimizer = self.optimizer(self.model.parameters())
            loss = self.loss_module(prediction, y_train)
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=4.0)
            self.optimizer.step()

            total_loss += loss
            mean_batch_loss = total_loss/(i+1)

            predictions = torch.argmax(prediction, 1)
            batch_accuracy = accuracy_score(y_train, predictions)

            
            total_batch_accuracy += batch_accuracy 
            mean_bach_accuracy = total_batch_accuracy/(i+1)
            
        return mean_batch_loss.detach().numpy(), mean_bach_accuracy
        
    def evaluate(self, data, batch_size):

        self.data = data
        total_batch_accuracy = 0
        total_loss = 0
        self.model = self.model.eval()
            
        for i, batch in enumerate(DataLoader(self.data, batch_size)):
            x_test, y_test, masks = batch
            prediction = self.model(x_test.float(),masks)
            loss = self.loss_module(prediction, y_test)

            total_loss += loss
            mean_batch_loss = total_loss/(i+1)

            predictions = torch.argmax(prediction, 1)
            batch_accuracy = accuracy_score(y_test, predictions)


            total_batch_accuracy += batch_accuracy 
            mean_bach_accuracy = total_batch_accuracy/(i+1)

        return mean_batch_loss.detach().numpy(), mean_bach_accuracy
         
    def train(self,train_data,val_data,epochs=100,batch_size=128,verbose=True,return_loss=True,early_stopping=10):
               
        best_loss = np.inf
        stopping_steps = 0
        
        for epoch in range(epochs):
    
            loss, batch_accuracy = self.fit(train_data, batch_size, verbose)
            val_loss, val_batch_accuracy = self.evaluate(val_data, batch_size)
            
            self.training_loss.append(loss)
            self.validation_loss.append(val_loss)
           
            if val_loss < best_loss:
                best_loss = val_loss
                stopping_steps = 0
                
            stopping_steps+=1
            if stopping_steps>early_stopping:
                break
            
            print('\033[94m'+'Epoch: {}'.format(epoch)+'\033[0m'+' {}%'.format(round(batch_accuracy*100,3)),
                  '\033[91m'+'loss:'+'\033[0m'+' {}'.format(round(float(loss),4)),
                  '\033[94m Val:'+'\033[0m'+' {}'.format(round(val_batch_accuracy*100,3)),
                  '\033[91m'+'val_loss:'+'\033[0m'+' {}'.format(round(float(val_loss),4))
                 )
                                              

        if return_loss:
            return plt.plot(self.training_loss), plt.plot(self.validation_loss, plt.legend())
            
        
    def analyze(self,data,color='viridis'):
        self.data = data
        self.model = self.model.eval()
        
        for i, batch in enumerate(DataLoader(self.data, 100000)):
            x_test, y_test, masks = batch
            prediction = self.model(x_test.float(), masks)
            predictions = torch.argmax(prediction,1)
            accuracy = accuracy_score(predictions, y_test)
            print('\033[92m'+'\nAccuracy: '+'\033[0m',accuracy)
            print('\n'+'\033[0m'+         'Classification Report:\n')
            print(classification_report(predictions, y_test))
            print('\n'+'\033[0m'+         'Confusion Matrix:\n')
            cm = confusion_matrix(predictions, y_test)
            print('\033[91m'+'             Total lenght of Data:'+'\033[0m', len(y_test),'\n')
            print('\033[94m'+'             Class 0:'+'\033[0m',(torch.unique(y_test, return_counts=True)[1][0]).item())
            print('\033[94m'+'             Class 1:'+'\033[0m',(torch.unique(y_test, return_counts=True)[1][1]).item(),'\n'+'\033[0m')

            plot_confusion_matrix(cm,[0,1],color=color)

            print('\n')

            try:
                fpr, tpr, _ = roc_curve(predictions, y_test)
                roc_auc = roc_auc_score(predictions, y_test)
                plt.figure()
                plt.plot(fpr, tpr, label='(Area = {:.3f})'.format(roc_auc))
                plt.xlabel('False positive rate')
                plt.ylabel('True positive rate')
                plt.title('ROC curve')
                plt.legend(loc='best')
                plt.show()

            except:
                print('Something wrong')

In [16]:
import ray

In [17]:
ray.init()

/Users/abhijitdeshpande/opt/anaconda3/lib/python3.8/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


{'node_ip_address': '10.219.175.110',
 'raylet_ip_address': '10.219.175.110',
 'redis_address': '10.219.175.110:6379',
 'object_store_address': '/tmp/ray/session_2021-12-20_12-23-11_261583_4257/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-20_12-23-11_261583_4257/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-12-20_12-23-11_261583_4257',
 'metrics_export_port': 63578,
 'node_id': '93e66c35c892ef7f456884e80eec815e3dcd0662c1dc10125cb7bcbc'}

In [20]:
@ray.remote
def f(x):
    return x * x

futures = [f.remote(i) for i in range(4)]

In [28]:
import ray
ray.init()

@ray.remote
def f(x):
    return x * x

futures = [f.remote(i) for i in range(4)]
print(ray.get(futures)) # [0, 1, 4, 9]

RayTaskError(RayOutOfMemoryError): [36mray::f()[39m (pid=9753, ip=10.219.175.110)
ray._private.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node Abhijits-MacBook-Pro.local is used (7.65 / 8.0 GB). The top 10 memory consumers are:

PID	MEM	COMMAND
7677	0.41GiB	/Users/abhijitdeshpande/opt/anaconda3/bin/python -m ipykernel_launcher -f /Users/abhijitdeshpande/Li
9435	0.15GiB	/System/Library/Frameworks/WebKit.framework/Versions/A/XPCServices/com.apple.WebKit.WebContent.xpc/C
4022	0.13GiB	/System/Library/Frameworks/WebKit.framework/Versions/A/XPCServices/com.apple.WebKit.WebContent.xpc/C
710	0.09GiB	/Applications/Safari.app/Contents/MacOS/Safari
9755	0.07GiB	ray::IDLE
5081	0.07GiB	/Applications/Microsoft Teams.app/Contents/Frameworks/Microsoft Teams Helper (Renderer).app/Contents
9750	0.07GiB	ray::IDLE
9746	0.07GiB	/Users/abhijitdeshpande/opt/anaconda3/bin/python -u /Users/abhijitdeshpande/opt/anaconda3/lib/python
9752	0.07GiB	ray::IDLE
9757	0.07GiB	ray::IDLE

In addi

2021-12-20 12:25:40,314	ERROR worker.py:79 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::f() (pid=9757, ip=10.219.175.110)
ray._private.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node Abhijits-MacBook-Pro.local is used (7.65 / 8.0 GB). The top 10 memory consumers are:

PID	MEM	COMMAND
7677	0.41GiB	/Users/abhijitdeshpande/opt/anaconda3/bin/python -m ipykernel_launcher -f /Users/abhijitdeshpande/Li
9435	0.15GiB	/System/Library/Frameworks/WebKit.framework/Versions/A/XPCServices/com.apple.WebKit.WebContent.xpc/C
4022	0.13GiB	/System/Library/Frameworks/WebKit.framework/Versions/A/XPCServices/com.apple.WebKit.WebContent.xpc/C
710	0.09GiB	/Applications/Safari.app/Contents/MacOS/Safari
9755	0.07GiB	ray::IDLE
5081	0.07GiB	/Applications/Microsoft Teams.app/Contents/Frameworks/Microsoft Teams Helper (Renderer).app/Contents
9750	0.07GiB	ray::IDLE
9746	0.07GiB	/Users/abhijitdeshpande/opt/anaconda3/bin/python -u /Users/abhijitdeshpande/opt/anaconda3/l

In [33]:
import schedule